## Search for overlapping Sentinel-1 (S1) and Sentinel-2 (S2) in Fram Strait
---

### This notebook searches for overlapping Sentinel-1 (EW GRDM) and Sentinel-2 (L1C) data. The search is implemented using the 'sentinelSAT' package, which is a powerful search API for the Copernicus Scihub database.

#### **Requirements to run this script:**
1) have a user account on Copernicus Open Access Hub (Scihub): https://scihub.copernicus.eu/dhus/#/home
2) create a file named '.env' in the directory 'S1_ice_water_classification'. In this file, save your username and password for Copernicus SciHub in exactly this format:

> DHUS_USER="scihub_username" <br>
> DHUS_PASSWORD="scihub_password"

_**NOTE**: it is strongly recommended to add the .env file to the .gitignore, to avoid pushing your credentials to a public repository!_

---


#### Import some packages needed to run this code

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from dateparser import parse
import datetime
from pathlib import Path
from dotenv import load_dotenv
from loguru import logger
import os

#### Load environment variables from .env file, containing username and password for Copernicus Scihub

In [2]:
load_dotenv()
    
try:
    os.environ["DHUS_USER"]
except:
    logger.error("The environment variable 'DHUS_USER' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_USER' is not set!")

try:
    os.environ["DHUS_PASSWORD"]
except:
    logger.error("The environment variable 'DHUS_PASSWORD' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_PASSWORD' is not set!")



#### Establish the connection to Copernicus Scihub

In [5]:
# establish connection to Copernicus Scihub
api = SentinelAPI(os.environ["DHUS_USER"], os.environ["DHUS_PASSWORD"])


#### Determine search criteria for Sentinel-1 imagery and perform search
In a first step, we search for Sentinel-1 EW GRDM products that contain our region of interest (ROI). 
The ROI for Fram Strait can be found in the 'rois' folder. We narrow the search down by the following criteria:
1) area (that is the ROI, given as GEOjson file)
2) time (start- and endtime)
3) platform and producttype: by default set to Sentinel-1 GRD

To create your own custom ROI, nagivate to : https://geojson.io/ and draw a polygon of your choice. Then, save the result in GEOjson file format. Upload it to the notebook and give the path to the new file in the variable 'area' (see below).

In [6]:
# search over Fram Strait for S1 imagery
# use custom geojson files containing search polygon
area = './rois/Fram_Strait.geojson'
polygon_path = Path(area).expanduser().absolute()

# set start- and endtime for search
starttime = "2020-03-01"
endtime = "2020-05-01"
    
starttime = parse(starttime, settings={"DATE_ORDER": "YMD"})
endtime = parse(endtime, settings={"DATE_ORDER": "YMD"})

# area relation for S1, 'Contains' returns all hits where the ROI polygon is entirely WITHIN the footprint of the S1
s1_area_relation = 'Contains'

# query for S1 products that match our search criteria
s1_products = api.query(
                geojson_to_wkt(read_geojson(polygon_path)),
                date=(starttime, endtime),
                platformname="Sentinel-1",
                producttype="GRD",
                area_relation=s1_area_relation,
                )

In [53]:
# create empty dictionary that will contain S1 identifiers as keys, and the overlapping S2 identifier(s) as values
s1_s2_overlap = {}

# iterate over query results to check for overlapping S2
for product in s1_products:

    # retrieve identifier and timestamp of the S1 product
    s1_identifier = s1_products[product]['identifier']
    s1_timestamp = s1_products[product]['beginposition']

    # look for S2 imagery taken 2 hours before or after S1 acquisition
    timedelta = datetime.timedelta(hours = 5) 
    s2_starttime = s1_timestamp - timedelta
    s2_endtime = s1_timestamp + timedelta
 
    # define cloud cover percentage range
    cloudcover = (0, 30)
    
    # area relation for S2, 'intersects' returns all hits where the ROI polygon INTERSECTS with the S1 footprint
    s2_area_relation = 'Intersects'

    # query for overlapping Sentinel-2 optical images
    s2_products = api.query(
                        area = geojson_to_wkt(read_geojson(polygon_path)),
                        date = (s2_starttime, s2_endtime),
                        platformname="Sentinel-2",
                        processinglevel="Level-1C",
                        cloudcoverpercentage=cloudcover,
                        area_relation=s2_area_relation,
                        )
    
    # if overlapping S2 scenes found, add the S1 id and corresponding S2 query results in dictionary
    if not len(s2_products) == 0:
        s2_id_list = []
        
        # loop over S2 search results and extract S2 product identifier
        for s2product in s2_products:
            s2_identifier = s2_products[s2product]['identifier']
            s2_id_list.append(s2_identifier)
        s1_s2_overlap[s1_identifier] = s2_id_list
            
print('Found', len(s1_s2_overlap), 'S1 scenes with overlapping S2 imagery.')


Found 3 S1 scenes with overlapping S2 imagery.


In [54]:
# save dictionary results to text files, these text files can then be used to download the S1 and S2 products on e.g. Creodias
for key, value in s1_s2_overlap.items(): 
    with open(f"./search_results_Fram_Strait/{key}.txt", 'w') as f:
        f.write('\n'.join(value))
        
print('Search results saved in "search_results_Fram_Strait" folder!')

Search results saved to txt files!
